# Visualize Laser Scan

SPDX-License-Identifier: MIT  
SPDX-FileCopyrightText: 2022 Daisuke Sato

https://github.com/Tiryoh/mini-pupper-jupyter-notebooks

## Set Network configuration

In [ ]:
# NETWORK_INTERFACE="eth0"
NETWORK_INTERFACE="wlan0"

import netifaces as ni
ip_address = ni.ifaddresses(NETWORK_INTERFACE)[ni.AF_INET][0]['addr']
print(ip_address)

In [ ]:
import os

# Set self ip address
os.environ['ROS_IP'] = ip_address

# Set ROS_MASTER_URI
ROS_MASTER_IP=ip_address
# ROS_MASTER_IP="192.168.10.14" # custom

os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROS_MASTER_IP)
print("ROS_MASTER_IP is {}".format(ROS_MASTER_IP))

### Check environment  

It should outputs like this:

```
192.168.10.14
http://192.168.10.14:11311
```

If the result is `http://localhost:11311`, re-run the above procedure.

In [ ]:
# Check Env
!echo $ROS_IP
!echo $ROS_MASTER_URI

## Import libraries

In [ ]:
from sidecar import Sidecar
# Using jupyros(jupyter-ros) 0.5.0 from conda for now
try:
    from jupyros import ros3d
except ImportError:
    from jupyros.ros1 import ros3d

# Check jupyros version
import jupyros
print(jupyros.__version__)

## Create View

The output should look like this:

<img src="https://i.gyazo.com/f5092c17bc8753f6c278346e48901b96.png" width="400px" />

In [ ]:
grid = ros3d.GridModel()
grid.color = '#CCC'
viewer = ros3d.Viewer()
viewer.layout.height= '1000px'
viewer.objects = [grid]

try:
    sc = Sidecar(title='LaserScan Visualize Output')
    with sc:
        display(viewer)
except:
    display(viewer)

## Subscribe Laser Scan

### Start lidar scan publish node

Before starting subscriber, we need to start the ROS node to publish camera image.  
Open terminal and run `cd ~/dev/mini-pupper-jupyter-notebooks && roslaunch ROS/bridge.launch`

<img src="https://i.gyazo.com/5ee3827c610736138f3f808fdce1f972.png" width="50%">

### Check topic

If it shows `ERROR: Unable to communicate with master!` or `ERROR: Unknown topic /scans`, rerun the network configuration step.

In [ ]:
!rostopic info /scan

### View scan topic

The output should look like this:

<img src="https://i.gyazo.com/db3430f1648fc9a30df1facc87339f75.png" width="400px" />

Reference
* https://jupyter-ros.readthedocs.io/en/latest/reference.html

In [ ]:
ros_connection = ros3d.ROSConnection(url="ws://{}:9090".format(ROS_MASTER_IP))
tf_client = ros3d.TFClient(ros=ros_connection, fixed_frame="laser_frame")
laser_view = ros3d.LaserScan(topic="/scan", ros=ros_connection, tf_client=tf_client)
laser_view.point_size = 0.02
laser_view.static_color = "red"

viewer.objects = [grid, laser_view]

#### Debug

If the outputs shows no scan data, check the "rosbridge_websocket" and "tf2_web_publisher" nodes are alive, and rosbridge client count is not 0. It should be 1 or more.

The result of `rosnode info rosbridge_websocket` should be like this:

```
Node [/rosbridge_websocket]
Publications: 
 * /client_count [std_msgs/Int32]
 * /connected_clients [rosbridge_msgs/ConnectedClients]
 * /rosout [rosgraph_msgs/Log]
 * /tf2_web_republisher/cancel [actionlib_msgs/GoalID]
 * /tf2_web_republisher/goal [tf2_web_republisher/TFSubscriptionActionGoal]

Subscriptions: 
 * /scan [sensor_msgs/LaserScan]
 * /tf2_web_republisher/feedback [tf2_web_republisher/TFSubscriptionActionFeedback]

Services: 
 * /rosbridge_websocket/get_loggers
 * /rosbridge_websocket/set_logger_level
 ```

In [ ]:
!rosnode info rosbridge_websocket

The result of `rosnode info tf2_web_republisher` should be like this:

```
Node [/tf2_web_republisher]
Publications: 
 * /rosout [rosgraph_msgs/Log]
 * /tf2_web_republisher/feedback [tf2_web_republisher/TFSubscriptionActionFeedback]
 * /tf2_web_republisher/result [tf2_web_republisher/TFSubscriptionActionResult]
 * /tf2_web_republisher/status [actionlib_msgs/GoalStatusArray]

Subscriptions: 
 * /tf [tf2_msgs/TFMessage]
 * /tf2_web_republisher/cancel [actionlib_msgs/GoalID]
 * /tf2_web_republisher/goal [tf2_web_republisher/TFSubscriptionActionGoal]
 * /tf_static [unknown type]

Services: 
 * /republish_tfs
 * /tf2_web_republisher/get_loggers
 * /tf2_web_republisher/set_logger_level
```

In [ ]:
!rosnode info tf2_web_republisher

In [ ]:
import rospy
from jupyros import subscribe
from std_msgs.msg import Int32

rospy.init_node('rosbridge_subscribe_node')
subscribe('/client_count', Int32, lambda msg: print(msg))